In [6]:

# 1. Carga de librerías con optimizaciones de memoria
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD
import gc

# Configuración para evitar problemas de memoria
def configure_memory():
    # Liberar memoria no utilizada
    gc.collect()

    # Configurar crecimiento de memoria de GPU
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU memory growth enabled")
        except RuntimeError as e:
            print(f"Memory growth configuration error: {e}")

    # Usar mixed precision para ahorrar memoria
    try:
        from tensorflow.keras import mixed_precision
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print("Mixed precision enabled (float16)")
    except:
        print("Could not enable mixed precision")



In [ ]:
# 2. Definición de carga de datos optimizada

def get_data_loaders(batch_size=64, pretrained=False):
    # Cargar CIFAR-10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Reducir tamaño de imagen para modelos preentrenados
    # (224x224 es demasiado grande para Colab, usamos 128x128)
    target_size = 128 if pretrained else 32

    # Preprocesamiento usando tf.data para eficiencia de memoria
    def preprocess(x, y):
        # Convertir a float32
        x = tf.cast(x, tf.float32) / 255.0

        # Redimensionar si es necesario
        if pretrained and x.shape[0] != target_size:
            x = tf.image.resize(x, (target_size, target_size))

        # Normalización
        if pretrained:
            mean = tf.constant([0.485, 0.456, 0.406])
            std = tf.constant([0.229, 0.224, 0.225])
        else:
            mean = tf.constant([0.5, 0.5, 0.5])
            std = tf.constant([0.5, 0.5, 0.5])

        x = (x - mean) / std
        return x, y

    # Crear datasets con procesamiento eficiente
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, test_ds


In [ ]:
# 3. Modelo AlexNet optimizado para memoria

def AlexNetOptimized(num_classes=10):
    model = models.Sequential([
        # Capas de características (reducidas para ahorrar memoria)
        layers.Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D(pool_size=2),
        layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=2),
        layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=2),

        # Capas clasificadoras (reducidas para ahorrar memoria)
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model


In [ ]:
# 4. Funciones de entrenamiento y evaluación con checkpoints

def train_model(model, train_ds, criterion, optimizer, num_epochs=10):
    # Callbacks para mejor gestión de memoria
    callbacks = [
    ]

    # Compilar modelo
    model.compile(
        optimizer=optimizer,
        loss=criterion,
        metrics=['accuracy']
    )

    # Entrenamiento
    print("Starting training...")
    for epoch in range(num_epochs):
        # Ajustar tasa de aprendizaje
        if epoch > 0 and epoch % 5 == 0:
            optimizer.learning_rate = optimizer.learning_rate * 0.5
            print(f"Learning rate adjusted to: {optimizer.learning_rate.numpy()}")

        # Entrenar por una época
        history = model.fit(
            train_ds,
            epochs=1,
            callbacks=callbacks,
            verbose=1
        )

        # Liberar memoria después de cada época
        gc.collect()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {history.history['loss'][0]:.4f}")

def evaluate_model(model, test_ds):
    loss, accuracy = model.evaluate(test_ds, verbose=1)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

In [5]:
# 5. Entrenamiento y evaluación con control de memoria

def main():
    # Configurar memoria
    configure_memory()

    print("\n===== Entrenando AlexNet optimizado =====")

    # Obtener datos con batch pequeño
    train_ds, test_ds = get_data_loaders(batch_size=32)

    # Crear modelo optimizado
    model = AlexNetOptimized()

    # Parámetros de entrenamiento
    criterion = 'sparse_categorical_crossentropy'
    optimizer = SGD(learning_rate=0.01, momentum=0.9)

    # Entrenar y evaluar
    train_model(model, train_ds, criterion, optimizer, num_epochs=10)
    evaluate_model(model, test_ds)

    # Liberar memoria antes de salir
    del model
    gc.collect()
    tf.keras.backend.clear_session()

if __name__ == "__main__":
    main()

GPU memory growth enabled
Mixed precision enabled (float16)

===== Entrenando AlexNet optimizado =====


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Starting training...
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.3048 - loss: 1.8582
Epoch 1/10, Loss: 1.6185
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.5435 - loss: 1.2643
Epoch 2/10, Loss: 1.2035
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6232 - loss: 1.0619
Epoch 3/10, Loss: 1.0297
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6697 - loss: 0.9434
Epoch 4/10, Loss: 0.9192
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6963 - loss: 0.8641
Epoch 5/10, Loss: 0.8525
Learning rate adjusted to: 0.004999999888241291
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7512 - loss: 0.7231
Epoch 6/10, Loss: 0.6893
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7694 - loss: 0.6542
Epoch 7/10, Loss: 0.6374
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7872 - loss: 0.6070
Epoch 8/10, Loss: 0.5932
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7990 - loss: 0.5767
Epoch 9/10, Loss: 0.5636
1563/1563 ━━